# Auto Annotations


In [1]:
cd ..

/Users/yanndubois/Desktop/GitHub/alpaca_farm


In [2]:
from alpaca_farm.utils import jload
from alpaca_farm.auto_annotations.pairwise_annotators import PairwiseAutoAnnotator, SinglePairwiseAutoAnnotator

The key object that you need for automatic annotations (both for training or for eval) is our `PairwiseAutoAnnotator`. By default the annotator is the pool from Alpaca Farm, for simplicity let's use a single annotator `test` for now.

For details about annotators including how to extend them, refer to the [Configuring annotators](#Configuring-annotators) section of this notebook.

In [3]:
annotator = PairwiseAutoAnnotator(annotators_config="annotators/test/configs.yaml")

## Annotating paired outputs

Now let's annotate some pairwise preference. All we need is some data. The annotator takes in either list of dictionaries, or pandas dataframes. The keys of dictionaries (or columns in dataframe) need to always contain `instruction` and `input` which defines the instruction. The annotator also needs a pair of outputs to compare, if you have a sequence of such pairs under the keys `output_1` and `output_2` then you can directly use `annotate_pairs`.

In [25]:
outputs_pairs = jload("examples/data/outputs_pairs.json")[:6]
print("Example of paired output:\n")
outputs_pairs[-1:]

Example of paired output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Hey everyone! \n\nI'm hosting a dinner party this Friday night and I'd love for all of you to come over. We'll have a delicious spread of food and some great conversations. \n\nLet me know if you can make it - I'd love to see you all there!\n\nCheers,\n[Your Name]"}]

In [26]:
annotated = annotator.annotate_pairs(outputs_pairs)

In [27]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Hey everyone! \n\nI'm hosting a dinner party this Friday night and I'd love for all of you to come over. We'll have a delicious spread of food and some great conversations. \n\nLet me know if you can make it - I'd love to see you all there!\n\nCheers,\n[Your Name]",
  'annotator': 'davinci003_3',
  'preference': 2.0}]

Here we see that the annotations adds two keys:
- `'preference'`: the index of the preferred output, here `preference=2.0` so `output_2` is prefered
- `'annotator'`: the name of the simulated annotator as found in the `annotators_config`

`annotate_pairs` is the main function and should be used if you have paired outputs to annotate. In many usecases, however, you  will outputs in different formats. In the following we discuss two additional helper function `annotate_head2head` and `annotate_samples` which are paticularly well suited for the typical format during evaluation and training respectively. Both functions call `annotate_pairs` under the hood after a reformatting step.

## Evaluation
For evaluation we need two components:
- outputs from the model we want to compare
- outputs on the same examples from the baseline model

Often case both of those components will be in a different list of dictionary (one list for each model). In this case all dictionaries need to contain an `output` column. Let us load such data from our simulated RLHF model and text-davinci-003 baseline.

In [28]:
outputs_baseline = jload("examples/data/outputs_baseline.json")[:6]
print("Example of baseline output:\n")
outputs_baseline[-1:]

Example of baseline output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]"}]

In [29]:
outputs_rlhf = jload("examples/data/outputs_rlhf.json")[:6]
print("Example of rlhf output:\n")
outputs_rlhf[-1:]

Example of rlhf output:



[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]"}]

The annotator's function of interest when we have two sequences of outputs is `annotate_head2head`.

In [30]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

In [31]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]",
  'annotator': 'davinci003_3',
  'preference': 2.0}]

We see that the format of the output is the same as before. Here `preference` indicates that the simulator prefered the output `output_2`, which corresponds to our RLHF model.

## Training
For training we typically have multiple outputs for each instruction which are sampled from the same (SFT) model as seen in the following data:

In [32]:
outputs_samples = jload("examples/data/multisamples_sft.json")[:3]
print("Example of different sampled outputs from SFT:\n")
outputs_samples

Example of different sampled outputs from SFT:



[{'instruction': 'Why is it important to save money?',
  'input': '',
  'output': 'Saving money is important because it allows you to have financial security in case of emergencies; to be able to afford the things you need; to have money for retirement; and to be able to afford the lifestyle you want. Saving money can also help you build wealth and reach your financial goals. It is important to develop the habit of regularly putting money aside for savings.'},
 {'instruction': 'Why is it important to save money?',
  'input': '',
  'output': 'It is important to save money because it can help create a financial cushion in case of an emergency, allow you to pursue goals, and give you more financial security. Having a savings can also give you the opportunity to make more money by investing it. Saving money can also help you prepare for big expenditures, such as buying a car or a house.'},
 {'instruction': 'Why is it important to save money?',
  'input': '',
  'output': 'Saving money is im

In this case, you can use the annotato's `annotate_samples`, which first samples pairs of outputs that have the same instruction/input and then annotate those.

In [33]:
annotated = annotator.annotate_samples(outputs_samples)

prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


In [34]:
annotated

[{'instruction': 'Why is it important to save money?',
  'input': '',
  'output_1': 'Saving money is important because it allows you to have financial security in case of emergencies; to be able to afford the things you need; to have money for retirement; and to be able to afford the lifestyle you want. Saving money can also help you build wealth and reach your financial goals. It is important to develop the habit of regularly putting money aside for savings.',
  'output_2': 'It is important to save money because it can help create a financial cushion in case of an emergency, allow you to pursue goals, and give you more financial security. Having a savings can also give you the opportunity to make more money by investing it. Saving money can also help you prepare for big expenditures, such as buying a car or a house.',
  'annotator': 'davinci003_3',
  'preference': 1.0}]

By default there will only be one pair per instruction,input. If you use `is_unique_instructions=False` then you will get as many pairs as outputs.    

## Going further

### Adding noise

During training we typically flip the the label with probability 0.25 to emulate the variability of human annotations. To do so you can either:
- initialize the annotator with `PairwiseAutoAnnotator(p_label_flip=0.25)`
- set the noise of an initialized annotator `annotator.set_noise(p_label_flip=0.25)` 
- give the noise to annotate_samples `annotate_samples(..., p_label_flip=0.25)` 

In [14]:
annotated = annotator.annotate_samples(outputs_samples, p_label_flip=0.5)

In [15]:
annotated

[{'instruction': 'Why is it important to save money?',
  'input': '',
  'output_1': 'Saving money is important because it allows you to have financial security in case of emergencies; to be able to afford the things you need; to have money for retirement; and to be able to afford the lifestyle you want. Saving money can also help you build wealth and reach your financial goals. It is important to develop the habit of regularly putting money aside for savings.',
  'output_2': 'It is important to save money because it can help create a financial cushion in case of an emergency, allow you to pursue goals, and give you more financial security. Having a savings can also give you the opportunity to make more money by investing it. Saving money can also help you prepare for big expenditures, such as buying a car or a house.',
  'annotator': 'davinci003_3',
  'preference': 2.0}]

### Cost & time efficiency: avoiding duplicate annotation
Often time you will find yourself reannotating examples that you already annotated. This is particularly true if you sample many times from the same model (eg to get pairwise preferences for training or to evaluate best-of-n) or if the instructions you are considering require short outputs => many models might give the same exact answer.

This means that you have to spend unecessary money and time. Thankfully `PairwiseAutoAnnotator` stores previous annotations and will reuse those. For example, let us reannotate the previous evaluation

In [16]:
annotated = annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

In [17]:
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]",
  'annotator': 'davinci003_3',
  'preference': 2.0}]

We now get the annotations without actually having to reannotate any example.
By default, this the previous annotations are only stored in memory. If you want those to be stateful, you need to either give `saving_path` to the constructor of `PairwiseAutoAnnotator` (automatically save and loads) or manually call `save` and `load`.
Here's an example

In [18]:
saving_path = "examples/data/annotations.json"
annotator.save(saving_path) # manually save the previous annotation
new_annotator = PairwiseAutoAnnotator(annotators_config="annotators/test/configs.yaml",
                                     saving_path=saving_path # will automatically load and save future annotations
                                        )

In [19]:
annotated = new_annotator.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)
annotated[-1:]

[{'instruction': 'If you could help me write an email to my friends inviting them to dinner on Friday, it would be greatly appreciated.',
  'input': '',
  'output_1': "Dear Friends, \r\n\r\nI hope this message finds you well. I'm excited to invite you to dinner on Friday. We'll meet at 7:00 PM at [location]. I look forward to seeing you there. \r\n\r\nBest,\r\n[Name]",
  'output_2': "Dear Friends,\n\nI'm planning a dinner party this Friday and I'd love for you to join me! It's going to be a casual night of great food, interesting conversations, and lots of laughs.\n\nI'm looking forward to seeing you there. Let me know if you have any dietary restrictions or if you have any questions.\n\nSincerely,\n[Your Name]",
  'annotator': 'davinci003_3',
  'preference': 2.0}]

We suggest that you simply set `saving_path=...` when initializing the annotator and you will never have to deal with manual saving and loading.

### Evaluating win rates 

Let's show how to get win rates from annotations. 

In [5]:
outputs_baseline = jload("examples/data/outputs_baseline.json")
outputs_rlhf = jload("examples/data/outputs_rlhf.json")

In [6]:
annotator_pool = PairwiseAutoAnnotator()

In [7]:
to_annotate = annotator_pool.annotate_head2head(outputs_1=outputs_baseline, outputs_2=outputs_rlhf)

INFO:root:Annotating 15 examples with gpt4_1
INFO:root:Auto annotating 5 samples using gpt-4-0314.
INFO:root:Kwargs to completion: OpenAIDecodingArgumentsChat(max_tokens=1800, temperature=0.2, top_p=1.0, n=1, stream=False, stop=None, presence_penalty=0.0, frequency_penalty=0.0) {'max_tokens': 600, 'temperature': 1.0}
INFO:root:Decoding with OpenAI API model gpt-4-0314 and numproc == 5.
prompt_batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:56<00:00, 11.31s/it]
INFO:root:Annotating 15 examples with gpt4_2
INFO:root:Auto annotating 4 samples using gpt-4-0314.
INFO:root:Kwargs to completion: OpenAIDecodingArgumentsChat(max_tokens=1800, temperature=0.2, top_p=1.0, n=1, stream=False, stop=None, presence_penalty=0.0, frequency_penalty=0.0) {'max_tokens': 250, 'temperature': 1.0}
INFO:root:Decoding with OpenAI API model gpt-4-0314 and numproc == 5.
prompt_batches: 100%|███████████████████████████████████

ValueError: All placeholders should be repeated batch_size=1 times but Counter({'instruction': 5, 'input': 5, 'output_1': 5, 'output_2': 5}).

### Configuring annotators

The most important argument to `PairwiseAutoAnnotator` is `annotators_config` which defines the pool of annotators. We provide the following options out of the box:
- `annotators_config="annotators/annotator_pool_v0/configs.yaml"`: ApacaFarm's default annotator pool
- `annotators_config="annotators/greedy_gpt4/configs.yaml"`: Greedy GPT4 annotator
- `annotators_config="annotators/test/configs.yaml"`: a faster text-davinci-003 annotator useful for testing

Here's the desciption of `annotators_config` from the docstring:
```
annotators_config : Path or list of dict
    A dictionary or path to a yaml file containing the configuration for the pool of annotators. The keys in the
        fist dictionary should be the annotator's name, and the value should be a dictionary of the annotator's
        configuration which should have the following keys:
        - prompt_templates (dict): a dictionary of prompts or path to the prompts.
        - fn_decoder (str): function in `alpaca_farm.auto_annotations.pairwise_annotators.decoders.py` for completions.
        - decoder_kwargs (dict): kwargs for fn_decode. E.g. model_name, max_completions_tokens
        - other kwargs to `SinglePairwiseAutoAnnotator`
```

And here is config `"annotators/test/configs.yaml"` of the annotator we used above. 

In [20]:
!cat src/alpaca_farm/auto_annotations/annotators/test/configs.yaml

davinci003_3 : # text-davinci-003_v1_b5-pairwise_temp=1.0
  prompt_templates:
    with_inputs: "annotators/annotator_pool_v0/text_b5_with_inputs.txt"
    without_inputs: "annotators/annotator_pool_v0/text_b5_without_inputs.txt"
  fn_decoder: "openai_completions"
  decoder_kwargs:
    model_name: "text-davinci-003"
    max_tokens: 200
    temperature: 0.0
  outputs_to_match:
    1: '\n\(a\)'
    2: '\n\(b\)'
  batch_size: 5

For more information see the docstrings of the following classes (eg uncomment and run the following)

In [21]:
# from alpaca_farm.auto_annotations.pairwise_annotators import SinglePairwiseAutoAnnotator
# PairwiseAutoAnnotator?

In [22]:
# SinglePairwiseAutoAnnotator?

If you want to make a new annotator, change the prompt and or the configs above. See `annotators/annotator_pool_v0/...txt` for the actual prompts we used. To see the prompt used above (which is part of our pool of prompts) uncomment and run the following line.

In [23]:
#!cat src/alpaca_farm/auto_annotations/annotators/annotator_pool_v0/text_b5_without_inputs.txt